In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
pip install pytorch-tabular


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [68]:
# === Load obfuscated data and classify ===
###############################################################################################################################
###############################################################################################################################
#####################################################ExtraTreesClassifier###############################################
###############################################################################################################################
###############################################################################################################################

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# === Load Obfuscated Dataset ===
df = pd.read_csv("NEWobfuscated_dataset.csv")

# === Prepare Features and Labels ===
X = df.drop(columns=['Label'], errors='ignore').select_dtypes(include=[np.number])
y = df['Label']

# === Random Projection Compression and Reconstruction ===
d = X.shape[1]
d_proj = int(0.3 * d)
R = np.random.normal(size=(d_proj, d))
X_proj = X.to_numpy() @ R.T
X_rec = X_proj @ np.linalg.pinv(R).T
X = pd.DataFrame(X_rec, columns=X.columns)

# === Flip 10% of Labels ===
flip_indices = np.random.choice(len(y), size=int(0.1 * len(y)), replace=False)
y.iloc[flip_indices] = 1 - y.iloc[flip_indices]

# === Replace NaNs and Scale ===
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)
X = StandardScaler().fit_transform(X)

# === Train/Test Split (Stratified) ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# === Train Extra Trees Classifier ===
model = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# === Evaluate ===
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))


<ipython-input-68-afbdf850276d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.iloc[flip_indices] = 1 - y.iloc[flip_indices]


✅ Accuracy: 0.5026666666666667
✅ F1 Score: 0.33451641526175685

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1492
           1       0.50      1.00      0.67      1508

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.34      3000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [69]:
###############################################################################################################################
###############################################################################################################################
#####################################################LightGBM with imbalance handling###############################################
###############################################################################################################################
###############################################################################################################################
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
import lightgbm as lgb

# === Load obfuscated dataset ===
df = pd.read_csv("NEWobfuscated_dataset.csv")

# === Keep only binary labels ===
df = df[df['Label'].isin([0, 1])]
X = df.drop(columns=['Label'], errors='ignore').select_dtypes(include=[np.number])
y = df['Label']

# === Drop constant columns ===
X = X.loc[:, (X != X.iloc[0]).any()]

# === Scale and clean ===
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)
X_scaled = StandardScaler().fit_transform(X)

# === Train/Test Split (Stratified) ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, stratify=y, random_state=42
)

# === LightGBM with imbalance handling ===
model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    is_unbalance=True,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# === Evaluation ===
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 3500, number of negative: 3500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 343
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 74
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
✅ Accuracy: 0.5033333333333333
✅ F1 Score: 0.503325386539518

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50      1500
           1       0.50      0.51      0.51      1500

    accuracy                           0.50      3000
   macro avg       0.50      0.50      0.50      3000
weighted avg       0.50      0.50      0.50      3000



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [35]:
###############################################################################################################################
###############################################################################################################################
#####################################################XGBoost =###############################################
###############################################################################################################################
###############################################################################################################################
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from collections import Counter

# === Load and preprocess ===
df = pd.read_csv("NEWobfuscated_dataset.csv")
df = df[df['Label'].isin([0, 1])]
X = df.drop(columns=['Label'], errors='ignore').select_dtypes(include=[np.number])
y = df['Label']
X = X.loc[:, (X != X.iloc[0]).any()]
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)
X = StandardScaler().fit_transform(X)

# === Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# === Compute class weights ===
counter = Counter(y_train)
scale_pos_weight = counter[0] / counter[1]  # balancing classes

# === XGBoost Classifier with tuning and early stopping ===
model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=scale_pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],

    verbose=False
)

# === Evaluate ===
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:07:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Accuracy: 0.5063333333333333
✅ F1 Score: 0.5062015988932158

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.49      0.50      1500
           1       0.51      0.52      0.51      1500

    accuracy                           0.51      3000
   macro avg       0.51      0.51      0.51      3000
weighted avg       0.51      0.51      0.51      3000



In [22]:
###############################################################################################################################
###############################################################################################################################
#############################################ensambel= =###############################################
###############################################################################################################################
###############################################################################################################################
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from collections import Counter

# === Load and preprocess ===
df = pd.read_csv("/content/NEWobfuscated_dataset (1).csv")
df = df[df['Label'].isin([0, 1])]
X = df.drop(columns=['Label'], errors='ignore').select_dtypes(include=[np.number])
y = df['Label']
X = X.loc[:, (X != X.iloc[0]).any()]
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)
X = StandardScaler().fit_transform(X)

# === Train/test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# === Class weight for XGBoost ===
counter = Counter(y_train)
scale_pos_weight = counter[0] / counter[1]

# === Define individual models ===
lgb_model = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    is_unbalance=True,
    random_state=42,
    n_jobs=-1
)

xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=scale_pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

mlp_model = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    max_iter=200,
    random_state=42
)

# === Ensemble with soft voting ===
ensemble = VotingClassifier(
    estimators=[
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('mlp', mlp_model)
    ],
    voting='soft'  # use probability averaging
)

# === Train ensemble ===
ensemble.fit(X_train, y_train)

# === Evaluate ===
y_pred = ensemble.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 3500, number of negative: 3500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 341
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 74
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:24:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Accuracy: 0.5063333333333333
✅ F1 Score: 0.5063043149980704

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.50      0.50      1500
           1       0.51      0.51      0.51      1500

    accuracy                           0.51      3000
   macro avg       0.51      0.51      0.51      3000
weighted avg       0.51      0.51      0.51      3000



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelSpreading

# === Load the obfuscated dataset ===
df = pd.read_csv("NEWobfuscated_dataset.csv")
df = df[df['Label'].isin([0, 1])]

# === Preprocess features ===
X = df.drop(columns=['Label'], errors='ignore').select_dtypes(include=[np.number])
X = X.loc[:, X.nunique() > 1]
y = df['Label'].astype(int)

# Clean NaNs/Infs
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)

# === Scale features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, stratify=y, random_state=42
)

# === 1. Logistic Regression ===
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print("🔵 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("F1 Score:", f1_score(y_test, y_pred_logreg, average='macro'))
print(classification_report(y_test, y_pred_logreg))

# === 2. k-Nearest Neighbors ===
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("\n🟢 k-Nearest Neighbors")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("F1 Score:", f1_score(y_test, y_pred_knn, average='macro'))
print(classification_report(y_test, y_pred_knn))

# === 3. Label Spreading (Semi-Supervised) ===
X_combined = np.vstack([X_train, X_test])
y_combined = np.concatenate([y_train, [-1] * len(y_test)])  # Simulate unlabeled test set

label_spread = LabelSpreading(kernel='knn', alpha=0.2)
label_spread.fit(X_combined, y_combined)
y_pred_ls = label_spread.transduction_[-len(y_test):]
print("\n🟣 Label Spreading")
print("Accuracy:", accuracy_score(y_test, y_pred_ls))
print("F1 Score:", f1_score(y_test, y_pred_ls, average='macro'))
print(classification_report(y_test, y_pred_ls))


🔵 Logistic Regression
Accuracy: 0.5063333333333333
F1 Score: 0.5062582297240548
              precision    recall  f1-score   support

           0       0.51      0.49      0.50      1500
           1       0.51      0.52      0.51      1500

    accuracy                           0.51      3000
   macro avg       0.51      0.51      0.51      3000
weighted avg       0.51      0.51      0.51      3000


🟢 k-Nearest Neighbors
Accuracy: 0.4836666666666667
F1 Score: 0.48316999302996844
              precision    recall  f1-score   support

           0       0.48      0.45      0.47      1500
           1       0.48      0.51      0.50      1500

    accuracy                           0.48      3000
   macro avg       0.48      0.48      0.48      3000
weighted avg       0.48      0.48      0.48      3000


🟣 Label Spreading
Accuracy: 0.49
F1 Score: 0.49287552911597965
              precision    recall  f1-score   support

           0       0.49      0.48      0.48      1500
           